In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.nn.utils.prune as prune

import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision import models
from torchmetrics import Accuracy

import quantus
import captum
from captum.attr import Saliency, IntegratedGradients, NoiseTunnel
from cleverhans.torch.attacks.projected_gradient_descent import (projected_gradient_descent)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import random
import copy
import gc
import math

import warnings
warnings.filterwarnings('ignore')

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
epochs = 120

batch_size = 128

classes = ('plane', 'car' , 'bird','cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
cifar_transforms = transforms.Compose([transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,])

# cifar_transforms = transforms.Compose([transforms.ToTensor(),normalize,])

In [7]:
train_dataset = torchvision.datasets.CIFAR10(root= './datasets', train = True, download=True, transform = cifar_transforms)
test_dataset =  torchvision.datasets.CIFAR10(root= './datasets', train = False, download=True, transform = cifar_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True) # num_workers=4,
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, pin_memory=True)

In [9]:
%run models.ipynb
%run utils.ipynb

In [14]:
model = vgg16()
learning_rate = 0.05
criterion = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9, weight_decay = 5e-4)

In [6]:
def train_model(model, epochs):
    model.train()
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [119]:
model_normal = train_model(model = model.to(device), epochs = epochs)

Epoch 1/120 - test accuracy: 25.93% and CE loss 1.91
Epoch 2/120 - test accuracy: 38.85% and CE loss 1.43
Epoch 3/120 - test accuracy: 50.82% and CE loss 1.32
Epoch 4/120 - test accuracy: 56.92% and CE loss 1.07
Epoch 5/120 - test accuracy: 64.45% and CE loss 0.98
Epoch 6/120 - test accuracy: 67.44% and CE loss 0.63
Epoch 7/120 - test accuracy: 72.58% and CE loss 1.03
Epoch 8/120 - test accuracy: 75.10% and CE loss 0.57
Epoch 9/120 - test accuracy: 74.16% and CE loss 0.61
Epoch 10/120 - test accuracy: 75.62% and CE loss 0.72
Epoch 11/120 - test accuracy: 77.35% and CE loss 0.61
Epoch 12/120 - test accuracy: 78.31% and CE loss 0.63
Epoch 13/120 - test accuracy: 76.15% and CE loss 0.63
Epoch 14/120 - test accuracy: 79.17% and CE loss 0.34
Epoch 15/120 - test accuracy: 82.22% and CE loss 0.46
Epoch 16/120 - test accuracy: 81.86% and CE loss 0.61
Epoch 17/120 - test accuracy: 82.60% and CE loss 0.69
Epoch 18/120 - test accuracy: 81.99% and CE loss 0.39
Epoch 19/120 - test accuracy: 82.54% 

In [120]:
# Model to GPU and eval mode.
model_normal.to(device)
model_normal.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_normal, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 86.99%


In [121]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "vgg_cifar.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_normal.state_dict(), f=model_save_path)

Saving the model: models\vgg_cifar.pth


# Adversarial Training

In [73]:
model = vgg16()
learning_rate = 0.05
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9, weight_decay = 5e-4)
eps= [0.01, 0.03, 0.06, 0.1, 0.3, 0.5]

In [8]:
def train_adv(model, epsilon, epochs):
    model.train()
    eps = epsilon
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            x_batch = projected_gradient_descent(model, x_batch, eps, eps/10, 40, np.inf)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [83]:
model_adversarial = train_adv(model=model.to(device),
                    epsilon = eps[0], 
                    epochs=epochs)

Epoch 1/100 - test accuracy: 21.61% and CE loss 1.97
Epoch 2/100 - test accuracy: 33.43% and CE loss 1.86
Epoch 3/100 - test accuracy: 34.10% and CE loss 1.79
Epoch 4/100 - test accuracy: 53.47% and CE loss 1.36
Epoch 5/100 - test accuracy: 61.15% and CE loss 1.08
Epoch 6/100 - test accuracy: 67.06% and CE loss 1.02
Epoch 7/100 - test accuracy: 69.84% and CE loss 0.79
Epoch 8/100 - test accuracy: 70.26% and CE loss 0.79
Epoch 9/100 - test accuracy: 74.50% and CE loss 0.68
Epoch 10/100 - test accuracy: 76.50% and CE loss 0.62
Epoch 11/100 - test accuracy: 77.97% and CE loss 0.81
Epoch 12/100 - test accuracy: 77.82% and CE loss 0.66
Epoch 13/100 - test accuracy: 75.05% and CE loss 0.64
Epoch 14/100 - test accuracy: 75.97% and CE loss 0.68
Epoch 15/100 - test accuracy: 79.45% and CE loss 0.62
Epoch 16/100 - test accuracy: 77.86% and CE loss 0.55
Epoch 17/100 - test accuracy: 80.09% and CE loss 0.72
Epoch 18/100 - test accuracy: 78.69% and CE loss 0.75
Epoch 19/100 - test accuracy: 82.92% 

In [85]:
# Model to GPU and eval mode.
model_adversarial.to(device)
model_adversarial.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_adversarial, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 86.29%


In [87]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "vgg_cifar_adv.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_adversarial.state_dict(), f=model_save_path)

Saving the model: models\vgg_cifar_adv.pth


# L1 Unstructured

In [21]:
model = vgg16()
epochs = 100
criterion = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05, momentum = 0.9, weight_decay = 5e-4)

In [23]:
orig_params = count_params(model)
print(f"Unpruned VGG-16 model has {orig_params} trainable parameters")

Unpruned VGG-16 model has 15240903 trainable parameters


In [25]:
for layer, param in model.named_parameters():
    print(f"layer.name: {layer} & param.shape = {param.shape}")

layer.name: features.0.weight & param.shape = torch.Size([64, 3, 3, 3])
layer.name: features.0.bias & param.shape = torch.Size([64])
layer.name: features.2.weight & param.shape = torch.Size([64, 64, 3, 3])
layer.name: features.2.bias & param.shape = torch.Size([64])
layer.name: features.5.weight & param.shape = torch.Size([128, 64, 3, 3])
layer.name: features.5.bias & param.shape = torch.Size([128])
layer.name: features.7.weight & param.shape = torch.Size([128, 128, 3, 3])
layer.name: features.7.bias & param.shape = torch.Size([128])
layer.name: features.10.weight & param.shape = torch.Size([256, 128, 3, 3])
layer.name: features.10.bias & param.shape = torch.Size([256])
layer.name: features.12.weight & param.shape = torch.Size([256, 256, 3, 3])
layer.name: features.12.bias & param.shape = torch.Size([256])
layer.name: features.14.weight & param.shape = torch.Size([256, 256, 3, 3])
layer.name: features.14.bias & param.shape = torch.Size([256])
layer.name: features.17.weight & param.shap

In [27]:
for layer_name in model.state_dict().keys():
    print(layer_name, model.state_dict()[layer_name].shape)

features.0.weight torch.Size([64, 3, 3, 3])
features.0.bias torch.Size([64])
features.2.weight torch.Size([64, 64, 3, 3])
features.2.bias torch.Size([64])
features.5.weight torch.Size([128, 64, 3, 3])
features.5.bias torch.Size([128])
features.7.weight torch.Size([128, 128, 3, 3])
features.7.bias torch.Size([128])
features.10.weight torch.Size([256, 128, 3, 3])
features.10.bias torch.Size([256])
features.12.weight torch.Size([256, 256, 3, 3])
features.12.bias torch.Size([256])
features.14.weight torch.Size([256, 256, 3, 3])
features.14.bias torch.Size([256])
features.17.weight torch.Size([512, 256, 3, 3])
features.17.bias torch.Size([512])
features.19.weight torch.Size([512, 512, 3, 3])
features.19.bias torch.Size([512])
features.21.weight torch.Size([512, 512, 3, 3])
features.21.bias torch.Size([512])
features.24.weight torch.Size([512, 512, 3, 3])
features.24.bias torch.Size([512])
features.26.weight torch.Size([512, 512, 3, 3])
features.26.bias torch.Size([512])
features.28.weight t

In [29]:
print(f"VGG-16 global sparsity = {compute_sparsity_vgg(model):.2f}%")

VGG-16 global sparsity = 0.00%


In [31]:
import torch.nn.utils.prune as prune
for name, module in model.named_modules():
    # prune 20% of weights/connections in for all hidden layaers-
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module = module, name = 'weight', amount = 0.2)
    
    # prune 10% of weights/connections for output layer-
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module = module, name = 'weight', amount = 0.1)

In [32]:
print(f"VGG-16 global sparsity = {compute_sparsity_vgg(model):.2f}%")

VGG-16 global sparsity = 19.65%


In [225]:
model_l1_unstructured = train_model(model = model.to(device), epochs = epochs)

Epoch 1/100 - test accuracy: 27.83% and CE loss 2.00
Epoch 2/100 - test accuracy: 34.03% and CE loss 1.58
Epoch 3/100 - test accuracy: 51.59% and CE loss 1.20
Epoch 4/100 - test accuracy: 61.68% and CE loss 1.13
Epoch 5/100 - test accuracy: 65.36% and CE loss 1.09
Epoch 6/100 - test accuracy: 69.52% and CE loss 0.93
Epoch 7/100 - test accuracy: 70.84% and CE loss 0.75
Epoch 8/100 - test accuracy: 75.78% and CE loss 0.93
Epoch 9/100 - test accuracy: 76.21% and CE loss 0.68
Epoch 10/100 - test accuracy: 77.65% and CE loss 0.52
Epoch 11/100 - test accuracy: 78.01% and CE loss 0.86
Epoch 12/100 - test accuracy: 80.66% and CE loss 0.45
Epoch 13/100 - test accuracy: 81.12% and CE loss 0.50
Epoch 14/100 - test accuracy: 79.91% and CE loss 0.37
Epoch 15/100 - test accuracy: 80.65% and CE loss 0.57
Epoch 16/100 - test accuracy: 79.09% and CE loss 0.54
Epoch 17/100 - test accuracy: 80.61% and CE loss 0.53
Epoch 18/100 - test accuracy: 82.94% and CE loss 0.67
Epoch 19/100 - test accuracy: 81.85% 

In [226]:
# Model to GPU and eval mode.
model_l1_unstructured.to(device)
model_l1_unstructured.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_l1_unstructured, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 86.19%


In [227]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "vgg_cifar_l1_unstructured.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_l1_unstructured.state_dict(), f=model_save_path)

Saving the model: models\vgg_cifar_l1_unstructured.pth


# Global Pruning

In [60]:
model = vgg16()
epochs = 100
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05, momentum = 0.9, weight_decay = 5e-4)

In [62]:
orig_params = count_params(model)
print(f"Unpruned VGG-16 model has {orig_params} trainable parameters")

Unpruned VGG-16 model has 15240906 trainable parameters


In [64]:
for layer, param in model.named_parameters():
    print(f"layer.name: {layer} & param.shape = {param.shape}")

layer.name: features.0.weight & param.shape = torch.Size([64, 3, 3, 3])
layer.name: features.0.bias & param.shape = torch.Size([64])
layer.name: features.2.weight & param.shape = torch.Size([64, 64, 3, 3])
layer.name: features.2.bias & param.shape = torch.Size([64])
layer.name: features.5.weight & param.shape = torch.Size([128, 64, 3, 3])
layer.name: features.5.bias & param.shape = torch.Size([128])
layer.name: features.7.weight & param.shape = torch.Size([128, 128, 3, 3])
layer.name: features.7.bias & param.shape = torch.Size([128])
layer.name: features.10.weight & param.shape = torch.Size([256, 128, 3, 3])
layer.name: features.10.bias & param.shape = torch.Size([256])
layer.name: features.12.weight & param.shape = torch.Size([256, 256, 3, 3])
layer.name: features.12.bias & param.shape = torch.Size([256])
layer.name: features.14.weight & param.shape = torch.Size([256, 256, 3, 3])
layer.name: features.14.bias & param.shape = torch.Size([256])
layer.name: features.17.weight & param.shap

In [66]:
print(f"VGG-16 global sparsity = {compute_sparsity_vgg(model):.2f}%")

VGG-16 global sparsity = 0.00%


In [68]:
parameters_to_prune = (
    (model.features[0], 'weight'),
    (model.features[2], 'weight'),
    (model.features[5], 'weight'),
    (model.features[7], 'weight'),
    (model.features[10], 'weight'),
    (model.features[12], 'weight'),
    (model.features[14], 'weight'),
    (model.features[17], 'weight'),
    (model.features[19], 'weight'),
    (model.features[21], 'weight'),
    (model.features[24], 'weight'),
    (model.features[26], 'weight'),
    (model.features[28], 'weight'),
    (model.classifier[1], 'weight'),
    (model.classifier[4], 'weight'),
    (model.classifier[6], 'weight')
)

prune_rates_global = [0.2, 0.3, 0.4, 0.5, 0.6]

In [10]:
def train_global_pruned(model, epochs):
    for iter_prune_round in range(1):
        print(f"\n\nIterative Global pruning round = {iter_prune_round + 1}")
        
        # Prune layer-wise in a structured manner-
        prune.global_unstructured(
            parameters_to_prune,
            pruning_method = prune.L1Unstructured,
            amount = prune_rates_global[iter_prune_round]
            
        )
    
        # Print current global sparsity level-
        print(f"VGG global sparsity = {compute_sparsity_vgg(model):.2f}%")
        
        
        # Fine-training loop-
        print("\nFine-tuning pruned model to recover model's performance\n")
        model.train()
        for epoch in range(epochs):
            for x_batch, y_batch in train_dataloader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                output = model(x_batch)
                loss = criterion(output, y_batch)
                loss.backward()
                optimizer.step()
    
            # Evaluate model!
            if epochs%10==0:
                predictions, labels = evaluate_model(model, test_dataloader, device)
                test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
                print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [236]:
model_global = train_global_pruned(model = model.to(device), epochs = epochs)



Iterative Global pruning round = 1
VGG global sparsity = 20.00%

Fine-tuning pruned model to recover model's performance

Epoch 1/100 - test accuracy: 25.65% and CE loss 1.84
Epoch 2/100 - test accuracy: 33.63% and CE loss 1.75
Epoch 3/100 - test accuracy: 44.22% and CE loss 1.71
Epoch 4/100 - test accuracy: 57.97% and CE loss 1.02
Epoch 5/100 - test accuracy: 66.98% and CE loss 0.96
Epoch 6/100 - test accuracy: 69.85% and CE loss 1.02
Epoch 7/100 - test accuracy: 71.94% and CE loss 0.88
Epoch 8/100 - test accuracy: 73.74% and CE loss 0.68
Epoch 9/100 - test accuracy: 72.61% and CE loss 0.78
Epoch 10/100 - test accuracy: 74.89% and CE loss 0.53
Epoch 11/100 - test accuracy: 76.51% and CE loss 0.55
Epoch 12/100 - test accuracy: 78.16% and CE loss 0.70
Epoch 13/100 - test accuracy: 79.52% and CE loss 0.53
Epoch 14/100 - test accuracy: 77.77% and CE loss 0.67
Epoch 15/100 - test accuracy: 81.76% and CE loss 0.60
Epoch 16/100 - test accuracy: 81.34% and CE loss 0.56
Epoch 17/100 - test a

In [237]:
# Model to GPU and eval mode.
model_global.to(device)
model_global.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_global, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 83.52%


In [238]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "vgg_cifar_global.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_global.state_dict(), f=model_save_path)

Saving the model: models\vgg_cifar_global.pth


# Layered Structure

In [72]:
model = vgg16()
epochs = 100
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05, momentum = 0.9, weight_decay = 5e-4)

In [74]:
orig_params = count_params(model)
print(f"Unpruned VGG-16 model has {orig_params} trainable parameters")

Unpruned VGG-16 model has 15240906 trainable parameters


In [76]:
for layer, param in model.named_parameters():
    print(f"layer.name: {layer} & param.shape = {param.shape}")

layer.name: features.0.weight & param.shape = torch.Size([64, 3, 3, 3])
layer.name: features.0.bias & param.shape = torch.Size([64])
layer.name: features.2.weight & param.shape = torch.Size([64, 64, 3, 3])
layer.name: features.2.bias & param.shape = torch.Size([64])
layer.name: features.5.weight & param.shape = torch.Size([128, 64, 3, 3])
layer.name: features.5.bias & param.shape = torch.Size([128])
layer.name: features.7.weight & param.shape = torch.Size([128, 128, 3, 3])
layer.name: features.7.bias & param.shape = torch.Size([128])
layer.name: features.10.weight & param.shape = torch.Size([256, 128, 3, 3])
layer.name: features.10.bias & param.shape = torch.Size([256])
layer.name: features.12.weight & param.shape = torch.Size([256, 256, 3, 3])
layer.name: features.12.bias & param.shape = torch.Size([256])
layer.name: features.14.weight & param.shape = torch.Size([256, 256, 3, 3])
layer.name: features.14.bias & param.shape = torch.Size([256])
layer.name: features.17.weight & param.shap

In [78]:
print(f"VGG-16 global sparsity = {compute_sparsity_vgg(model):.2f}%")

VGG-16 global sparsity = 0.00%


In [12]:
def train_layered_pruned(model, epochs):
    for iter_prune_round in range(1):
        print(f"\n\nIterative Global pruning round = {iter_prune_round + 1}")
        
        # Prune layer-wise in a structured manner-
        prune.ln_structured(model.features[0], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.features[2], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.features[5], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.features[7], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.features[10], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.features[12], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.features[14], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.features[17], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.features[19], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.features[21], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.features[24], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.features[26], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.features[28], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.classifier[1], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.classifier[4], name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.classifier[6], name = "weight", amount = 0.1, n = 2, dim = 0)
        
        # Print current global sparsity level-
        print(f"VGG global sparsity = {compute_sparsity_vgg(model):.2f}%")
        
        
        # Fine-training loop-
        print("\nFine-tuning pruned model to recover model's performance\n")
        model.train()
        for epoch in range(epochs):
            for x_batch, y_batch in train_dataloader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                output = model(x_batch)
                loss = criterion(output, y_batch)
                loss.backward()
                optimizer.step()
    
            # Evaluate model!
            if epochs%10==0:
                predictions, labels = evaluate_model(model, test_dataloader, device)
                test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
                print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [245]:
model_layered_structured = train_layered_pruned(model = model.to(device), epochs = epochs)



Iterative Global pruning round = 1
VGG global sparsity = 9.98%

Fine-tuning pruned model to recover model's performance

Epoch 1/100 - test accuracy: 19.48% and CE loss 1.81
Epoch 2/100 - test accuracy: 34.65% and CE loss 1.74
Epoch 3/100 - test accuracy: 43.64% and CE loss 1.52
Epoch 4/100 - test accuracy: 53.53% and CE loss 1.43
Epoch 5/100 - test accuracy: 62.37% and CE loss 1.08
Epoch 6/100 - test accuracy: 69.18% and CE loss 0.87
Epoch 7/100 - test accuracy: 69.92% and CE loss 0.88
Epoch 8/100 - test accuracy: 74.65% and CE loss 0.63
Epoch 9/100 - test accuracy: 74.63% and CE loss 0.85
Epoch 10/100 - test accuracy: 74.01% and CE loss 0.70
Epoch 11/100 - test accuracy: 75.85% and CE loss 0.67
Epoch 12/100 - test accuracy: 78.31% and CE loss 0.83
Epoch 13/100 - test accuracy: 80.22% and CE loss 0.50
Epoch 14/100 - test accuracy: 77.32% and CE loss 0.68
Epoch 15/100 - test accuracy: 79.81% and CE loss 0.77
Epoch 16/100 - test accuracy: 81.33% and CE loss 0.56
Epoch 17/100 - test ac

In [246]:
# Model to GPU and eval mode.
model_layered_structured.to(device)
model_layered_structured.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_layered_structured, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 85.68%


In [247]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "vgg_cifar_structured.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_layered_structured.state_dict(), f=model_save_path)

Saving the model: models\vgg_cifar_structured.pth
